In [4]:
library(dplyr)
library(lme4)
library(car) # for VIF calculation
library(tidyverse)
library(boot)
library(modelsummary)
library(lmerTest)
library(ggeffects)
library(magrittr)
library(broom)
library(broom.mixed)
library(sjPlot)
library(sjmisc)
library(sjlabelled)
library(jtools)
library(stargazer)
set.seed(12696921)



Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 




# CORONA

In [5]:

CORONA_INTERIM_PATH <- "/m/cs/work/luongn1/digirhythm/data/interim/corona/"
CORONA_PROCESSED_PATH <- "/m/cs/work/luongn1/digirhythm/data/processed/corona/"


SIMILARITY_PATH <- "/m/cs/work/luongn1/digirhythm/data/processed/corona/similarity_matrix/"

# Read survey data
survey <- read.csv(paste0(CORONA_INTERIM_PATH, "survey_all.csv"))

# Filter out 'non-binary' gender
survey <- survey %>% filter(gender != 'non-binary')

# Read similarity data
sim_baseline <- read.csv(paste0(SIMILARITY_PATH, "si/similarity_baseline_4epochs.csv"), row.names = 1)

# Keep only necessary columns
IVs <- c("subject_id", "age", "gender", "occupation", "origin", "children_at_home", "BIG5_Extraversion", "BIG5_Agreeableness", "BIG5_Conscientiousness", "BIG5_Neuroticism", "BIG5_Openness", "MEQ")
demographics_df <- survey %>% select(all_of(IVs)) %>% drop_na()

# Calculate average similarity
avg_sim_baseline <- rowMeans(sim_baseline, na.rm = TRUE)
avg_sim_baseline <- data.frame(subject_id = rownames(sim_baseline), DV = avg_sim_baseline)

# Merge datasets
dataset <- merge(avg_sim_baseline, demographics_df, by = 'subject_id', all.x = TRUE)

# Define a function to extract the coefficients
boot_fn <- function(data, indices) {
  d <- data[indices, ]  # Extract the bootstrapped sample
  fit <- lm(DV ~ age + origin + occupation + children_at_home + MEQ, data = d)
  return(coef(fit))
}

# Regression analysis with bootstrapping
regression_analysis <- function(df, y, X) {
  df <- df %>% drop_na()
  model <- lm(as.formula(paste(y, "~", paste(X, collapse = "+"))), data = df)
  vif_values <- vif(model)
summ(model, scale=TRUE, vifs=TRUE, confint = TRUE, digits = 3)
}

# Run the analysis
regression_analysis(dataset, "DV", c("age", "origin", "gender", "occupation", "children_at_home", "MEQ"))


MODEL INFO:
Observations: 115
Dependent Variable: DV
Type: OLS linear regression 

MODEL FIT:
F(6,108) = 3.198, p = 0.006
R² = 0.151
Adj. R² = 0.104 

Standard errors: OLS
---------------------------------------------------------------------------
                           Est.     2.5%    97.5%    t val.       p     VIF
---------------------- -------- -------- -------- --------- ------- -------
(Intercept)               0.680    0.669    0.690   125.246   0.000        
age                       0.005    0.000    0.010     2.018   0.046   1.059
origin                   -0.017   -0.030   -0.004    -2.509   0.014   1.415
gender1                   0.005   -0.006    0.016     0.960   0.339   1.073
occupation                0.014    0.002    0.027     2.362   0.020   1.482
children_at_home          0.002   -0.003    0.007     0.934   0.352   1.016
MEQ                       0.007    0.002    0.012     2.633   0.010   1.033
--------------------------------------------------------------------

# MOMO

In [6]:
MOMO_INTERIM_PATH <- "/m/cs/work/luongn1/digirhythm/data/interim/momo/"
MOMO_PROCESSED_PATH <- "/m/cs/work/luongn1/digirhythm/data/processed/momo/"

MOMO_SIMILARITY_PATH <- "/m/cs/work/luongn1/digirhythm/data/processed/momo/similarity_matrix/"

# Read survey data
survey <- read.csv(paste0(MOMO_INTERIM_PATH, "survey_all.csv"))
# Relevel
survey$group <- as.factor(survey$group)
survey <- within(survey, group <- relevel(group, ref = 'mmm-control'))

# Read similarity data
sim_baseline <- read.csv(paste0(MOMO_SIMILARITY_PATH, "si/similarity_baseline_4epochs.csv"), row.names = 1)

# Keep only necessary columns
IVs <- c("user", "bg_age", "bg_sex", "children", "work", "group")
demographics_df <- survey %>% select(all_of(IVs)) %>% drop_na()

# Calculate average similarity
avg_sim_baseline <- rowMeans(sim_baseline, na.rm = TRUE)
avg_sim_baseline <- data.frame(user = rownames(sim_baseline), DV = avg_sim_baseline)

# Merge datasets
dataset <- merge(avg_sim_baseline, demographics_df, by = 'user', all.x = TRUE)

# Define a function to extract the coefficients
boot_fn <- function(data, indices) {
  d <- data[indices, ]  # Extract the bootstrapped sample
  fit <- lm(DV ~ age + origin + occupation + children_at_home + MEQ, data = d)
  return(coef(fit))
}

# Regression analysis with bootstrapping
regression_analysis <- function(df, y, X) {
  df <- df %>% drop_na()
  model <- lm(as.formula(paste(y, "~", paste(X, collapse = "+"))), data = df)
  vif_values <- vif(model)
summ(model, scale=TRUE, vifs=TRUE, confint = TRUE, digits = 3)
}

# Run the analysis
regression_analysis(dataset, "DV", c("bg_age", "bg_sex", "children", "work", "group"))


MODEL INFO:
Observations: 54
Dependent Variable: DV
Type: OLS linear regression 

MODEL FIT:
F(7,46) = 0.780, p = 0.607
R² = 0.106
Adj. R² = -0.030 

Standard errors: OLS
---------------------------------------------------------------------
                       Est.     2.5%   97.5%   t val.       p     VIF
------------------ -------- -------- ------- -------- ------- -------
(Intercept)           0.640    0.598   0.683   30.251   0.000        
bg_age                0.008   -0.006   0.022    1.198   0.237   1.130
bg_sex                0.007   -0.027   0.042    0.422   0.675   1.136
children             -0.011   -0.042   0.021   -0.670   0.506   1.304
work                 -0.002   -0.033   0.030   -0.104   0.918   1.492
groupmmm-bd          -0.037   -0.092   0.019   -1.336   0.188   2.043
groupmmm-bpd         -0.002   -0.052   0.048   -0.080   0.936   2.043
groupmmm-mdd         -0.001   -0.038   0.036   -0.043   0.966   2.043
-----------------------------------------------------------

In [23]:
table(survey$work_regular)
#colnames(survey)


 0  1 
51 27 

# Wellbeing ~ Regularity

In [16]:
create_formula <- function(dv, frequency) {
  formula_str <- sprintf(
    '%s ~ 1 + 
    baseline_similarity +
    steps.night.%s.sum.norm + steps.morning.%s.sum.norm + steps.afternoon.%s.sum.norm + steps.evening.%s.sum.norm +
    steps.total.norm +
    tst.norm.mean + 
    midsleep.norm.mean + 
    heart_rate_variability_avg.mean.norm + 
    age.norm + gender + occupation + origin +
    (1|subject_id)',
    dv, frequency, frequency, frequency, frequency
  )
  
  as.formula(formula_str)
}

frequency <- '7ds'

# Read survey data
data <- read.csv(paste0(CORONA_PROCESSED_PATH, sprintf("%s_regularity_wellbeing.csv", frequency) ))

# Fitting the model for y1 with a random intercept for 'subject'
# Define the formula
formula1 <- create_formula('PHQ', frequency)

formula2 <- create_formula('PSS', frequency)

formula3 <- create_formula('PSQI', frequency)


# Fit the linear mixed-effects model
fit1 <- lmer(formula1, data = data)
fit2 <- lmer(formula2, data = data)
fit3 <- lmer(formula3, data = data)

# Display the summary of the model fit
tab_model(fit1, fit2, fit3,
         show.r2 = TRUE,
    show.icc = FALSE,
    show.re.var = FALSE,
    emph.p = TRUE,
    file = sprintf("%s_wellbeing_reg.html", frequency))

class(fit1) <- "lmerMod"
stargazer(fit1, out='4.tex')

fixed-effect model matrix is rank deficient so dropping 1 column / coefficient

fixed-effect model matrix is rank deficient so dropping 1 column / coefficient

fixed-effect model matrix is rank deficient so dropping 1 column / coefficient




% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Wed, Apr 10, 2024 - 10:29:51
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{1}{c}{\textit{Dependent variable:}} \\ 
\cline{2-2} 
\\[-1.8ex] & PHQ \\ 
\hline \\[-1.8ex] 
 baseline\_similarity & $-$0.656 \\ 
  & (0.735) \\ 
  & \\ 
 steps.night.7ds.sum.norm & $-$1.005 \\ 
  & (1.382) \\ 
  & \\ 
 steps.morning.7ds.sum.norm & $-$1.330$^{***}$ \\ 
  & (0.513) \\ 
  & \\ 
 steps.afternoon.7ds.sum.norm & $-$0.495 \\ 
  & (0.443) \\ 
  & \\ 
 steps.total.norm & $-$0.957$^{***}$ \\ 
  & (0.249) \\ 
  & \\ 
 tst.norm.mean & $-$0.558 \\ 
  & (0.473) \\ 
  & \\ 
 midsleep.norm.mean & $-$0.886$^{**}$ \\ 
  & (0.396) \\ 
  & \\ 
 heart\_rate\_variability\_avg.mean.norm & $-$0.361$^{**}$ \\ 
  & (0.174) \\ 
  & \\ 
 age.norm & $-$0.504 \\ 
  & (0.786) \\ 
  & \\ 
 ge

In [14]:
labels = rev(c("Baseline similarity", "Steps:Night", "Steps:Morning", 
           "Steps:Afternoon", "Steps:Evening", "Total steps", "Total sleep time",
           "Midsleep", "HRV", "Age", "Gender", "Occupation", "Origin"))

fplot <- plot_models(
  fit1, fit2, fit3,
#  axis.labels = labels,
  m.labels = c("PHQ", "PSS", "PSQI"),
     p.shape = TRUE
)
save_plot('forestplot.png', fplot)

png 
  2

In [48]:
pearsonboot = function(var1, data, indices,...) # indices needs to be supplied to boot function
{
  d1 = data[var1][indices,]
  d2 = data[indices,]

  pear = cor(x = d1, y = d2, use = "pairwise.complete.obs", method = "pearson")
  pear = pear[2:length(pear)]

  return(pear)
}

cormat = data.frame(data$PSS, data$PHQ, data$PSQI, data$baseline_similarity)
boots = boot(data = cormat[c("data.PSQI", "data.baseline_similarity")], pearsonboot, R = 1000, var1 = "data.PSQI")
bootsCI = boot.ci(boots, type = "perc")
bootsCI = round(c(bootsCI$t0, bootsCI$percent[4],bootsCI$percent[5]), digits = 2)
bootsCI = paste0("$r$ = ", bootsCI[1], ", $CI_{95\\%}$ = [", bootsCI[2], ", ", bootsCI[3], "]")
EES = gsub("0*\\.",".", bootsCI)
EES

[1] "$r$ = -.06, $CI_{95\\%}$ = [-.13, .02]"

In [40]:
colnames(cormat)

[1] "data.PSS"                 "data.PHQ"                
[3] "data.PSQI"                "data.baseline_similarity"